# Auto-Analyser-UAT Stats
### Follow these steps to generate statistics of names tested against the auto analyser

## Setup the environment

In [ ]:
%run ./notebook-setup.ipynb

## Set the data set
#### Default: entire data set of results that haven't been seen
*Could be expanded upon to select other subsets of uat jobs / names*

In [ ]:
data_set = 'unseen'  # all, unseen
uat_type = 'uat_accuracy'  # uat_rejection, uat_accuracy, all

## Get the uat jobs included in the data set

In [ ]:
if data_set == 'unseen':
    if uat_type != 'all':
        uat_jobs = UatJobResult.get_jobs_with_unsent_results(uat_type=uat_type)
    else:
        uat_jobs = UatJobResult.get_jobs_with_unsent_results()
elif uat_type != 'all':
    uat_jobs = UatJobResult.get_finshed_jobs(uat_type=uat_type)
else:
    uat_jobs = UatJobResult.get_finshed_jobs()
uat_jobs

## Get the response time averages

In [ ]:
total_request_time = 0.0
count = 0
for job in uat_jobs:
    time = job.get_request_time_avg()
    print(time)
    total_request_time += time
    count += 1
total_request_time_avg = total_request_time/count
total_request_time_avg

## Get the pass rate

#### Overall

In [ ]:
total_pass_avgs = 0.0
count = 0
for job in uat_jobs:
    pass_avg = job.get_accuracy()
    print(pass_avg)
    total_pass_avgs += pass_avg
    count += 1
total_pass_rate = total_pass_avgs/count
total_pass_rate

In [ ]:
approved_names_total = []
approved_names_passed = []
rejected_names_total = []
rejected_names_passed = []
for job in uat_jobs:
    # total adds both failed and passed (otherwise it would include err'd names)
    if uat_type in ['all', 'uat_accuracy']:
        approved_names_total += job.get_names(name_state='APPROVED', result='FAILED')
        approved_names_total += job.get_names(name_state='APPROVED', result='PASSED')
        approved_names_passed += job.get_names(name_state='APPROVED', result='PASSED')
    rejected_names_total += job.get_names(name_state='REJECTED', result='FAILED')
    rejected_names_total += job.get_names(name_state='REJECTED', result='PASSED')
    rejected_names_passed += job.get_names(name_state='REJECTED', result='PASSED')

#### Approvals pass rate

In [ ]:
if approved_names_total:
    print(f'total: {len(approved_names_total)}')
    print(f'passed: {len(approved_names_passed)}')

    print(f'pass rate: {len(approved_names_passed)/len(approved_names_total)}')
else:
    print('No approvals associated with these results')

#### Rejections pass rate

In [ ]:
print(f'total: {len(rejected_names_total)}')
print(f'passed: {len(rejected_names_passed)}')

print(f'pass rate: {len(rejected_names_passed)/len(rejected_names_total)}')

## Get the failed names

In [ ]:
should_approve_failed_names = []
should_reject_failed_names = []
for job in uat_jobs:
    should_reject_failed_names += job.get_names(name_state='REJECTED', result='FAILED')
    if uat_type in ['all', 'uat_accuracy']:
        should_approve_failed_names += job.get_names(name_state='APPROVED', result='FAILED')

#### Approval failures

In [ ]:
for name in should_approve_failed_names:
    print(name.name)

#### Rejection failures

In [ ]:
for name in should_reject_failed_names:
    print(name.name)

## Save failed results to excel spreadsheet

In [ ]:
if should_approve_failed_names:
    write_to_excel(should_approve_failed_names, f'{uat_type}_failed_to_approve.xlsx')
write_to_excel(should_reject_failed_names, f'{uat_type}_failed_to_reject.xlsx')

## Get the err'd names

In [ ]:
errd_names = []
non_timeout_errs = []
for job in uat_jobs:
    errd_names += job.get_names(result='ERROR')
for name in errd_names:
    if name.auto_analyse_issue_type != '504':
        non_timeout_errs.append(name)
        print(name.name)

In [ ]:
write_to_excel(non_timeout_errs, f'{uat_type}_error_names.xlsx')

## Update jobs results sent

In [ ]:
for job in uat_jobs:
    job.results_sent = True
    job.save()
db.session.commit()

## Create MD file of notebook run
**NOTE:** save notebook (i.e. _cmd s_) now to have results show in markdown file

In [ ]:
%%javascript
IPython.notebook.kernel.execute('nb_name = "' + IPython.notebook.notebook_name + '"')

In [ ]:
md_name = nb_name[:-6]+'.md'

In [ ]:
%%bash -s "$nb_name" "$md_name"
jupyter nbconvert $1 --to markdown --output $2